<a href="https://colab.research.google.com/github/metehan-karaca/material_identification/blob/main/Camera_Feed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Step 1: Install the required packages
!pip install tensorflow huggingface_hub ipywidgets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 9.5 MB/s eta 0:00:00


In [2]:

# Import necessary libraries
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from huggingface_hub import hf_hub_download
from huggingface_hub import snapshot_download
from PIL import Image
import io
from huggingface_hub import from_pretrained_keras

In [6]:

# Step 2: Download the full model directory from HuggingFace

model = from_pretrained_keras("metehan-karaca/model-functionalApi3-e100")



Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

variables/variables.index:   0%|          | 0.00/4.89k [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.70k [00:00<?, ?B/s]

keras_metadata.pb:   0%|          | 0.00/45.2k [00:00<?, ?B/s]

fingerprint.pb:   0%|          | 0.00/58.0 [00:00<?, ?B/s]

variables.data-00000-of-00001:   0%|          | 0.00/4.09M [00:00<?, ?B/s]

saved_model.pb:   0%|          | 0.00/420k [00:00<?, ?B/s]

In [7]:


# Function to convert the JavaScript object into an image
def js_to_image(js_reply):
    """
    Converts the JavaScript object returned by the browser into an OpenCV image.
    """
    image_bytes = b64decode(js_reply.split(',')[1])
    img = Image.open(io.BytesIO(image_bytes))
    return img


class_names = ['Amoledscreen-1B', 'concrete-1A', 'concrete-1B', 'Cooper-1A',
               'Copper-1B', 'denimfabric-1A', 'denimfabric-1B', 'fabric-1A',
               'fabric-1B', 'faux-leather-1A', 'faux-leather-1B', 'Granite-1A',
               'Granite-1B', 'LCDscreen-1B', 'Paper(not scratched)-1B', 'Plastic1-A',
               'Plastic1-B', 'Plastic2-A', 'Plastic2-B', 'satinfabric-1A',
               'satinfabric-1B', 'Scratchedpaper-1B', 'Spandexfabric-1B',
               'Spandexfabric1-A', 'Steel-1A', 'Steel-1B', 'Steel-alloy mixture1-A',
               'Steel-alloy mixture1-B', 'Teflon-1A', 'Teflon-1B', 'velvetfabric1-A',
               'velvetrabric1-B', 'Wood- 3B', 'wood-1A', 'wood-1B', 'Wood-2 B', 'Wood-2A',
               'Wood-2B', 'Wood-3A', 'Wood-3B', 'Wood-4A', 'Wood-4B', 'Wood-5A', 'Wood-5B']

# Function to capture a video frame
def take_photo(filename='photo.jpg', quality=0.8):
    js = Javascript('''
        async function takePhoto(quality) {
            const div = document.createElement('div');
            const capture = document.createElement('button');
            capture.textContent = 'Capture';
            div.appendChild(capture);

            const video = document.createElement('video');
            video.style.display = 'block';
            const stream = await navigator.mediaDevices.getUserMedia({video: true});

            document.body.appendChild(div);
            div.appendChild(video);
            video.srcObject = stream;
            await video.play();

            // Resize the output to fit the video element.
            google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

            // Wait for Capture to be clicked.
            await new Promise((resolve) => capture.onclick = resolve);

            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            canvas.getContext('2d').drawImage(video, 0, 0);
            stream.getVideoTracks()[0].stop();
            div.remove();
            return canvas.toDataURL('image/jpeg', quality);
        }
    ''')
    display(js)
    data = eval_js('takePhoto({})'.format(quality))
    img = js_to_image(data)
    return img

# Capture image from digital microscope
img = take_photo() # This line will create a button in the notebook

# Preprocess the image

size = model.input_shape

img = img.resize((size[2],size[1]))

img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0  # Normalizing

# Make a prediction
predicted_index = np.argmax(predictions, axis=1)[0]
print(class_names[predicted_index])




<IPython.core.display.Javascript object>

Wood-2 B
